# Dynamic Lightweight Convolutions tests

And other things like Large Memory with Product Keys and such


In [1]:
import torch
import torch.nn as nn

import fairseq
import fairseq.models

In [2]:
from fairseq.models import lightconv_lm

In [3]:
lm = lightconv_lm.FairseqLanguageModel?

Init signature: lightconv_lm.FairseqLanguageModel(decoder)
Docstring:     
Base class for decoder-only models.

Args:
    decoder (FairseqDecoder): the decoder
Init docstring: Initializes internal Module state, shared by both nn.Module and ScriptModule.
File:           ~/venv3/lib/python3.7/site-packages/fairseq/models/fairseq_model.py
Type:           type
Subclasses:     RobertaModel, LightConvLanguageModel, TransformerLanguageModel, FConvLanguageModel


In [ ]:
lm = lightconv_lm.FairseqLanguageModel

In [4]:
lm = lightconv_lm.FairseqLanguageModel

In [5]:
lm = lightconv_lm.FairseqLanguageModel

### test of outer product of sets (all combinations)

[source](https://discuss.pytorch.org/t/how-to-create-a-combination-of-concatenations-with-two-tensors/28709)

In [6]:
X = torch.tensor([ [1,2,3], [4,5,6] ])
Y = torch.tensor([ [7, 8], [9, 10] ] )

In [7]:
print(X.shape,Y.shape)

torch.Size([2, 3]) torch.Size([2, 2])


In [8]:
len(X.shape)

2

In [9]:
X1 = X.unsqueeze(0)
Y1=Y.unsqueeze(1)
print(X1.shape,Y1.shape)

torch.Size([1, 2, 3]) torch.Size([2, 1, 2])


In [10]:
X2 = X1.repeat(Y.shape[0],1,1)
Y2 = Y1.repeat(1,X.shape[0],1)
print(X2.shape,X2.shape)

torch.Size([2, 2, 3]) torch.Size([2, 2, 3])


In [11]:
Z = torch.cat([X2,Y2],-1)
print(Z.shape)
Z = Z.view(-1,Z.shape[-1])
print(Z.shape)

torch.Size([2, 2, 5])
torch.Size([4, 5])


In [12]:
X2

tensor([[[1, 2, 3],
         [4, 5, 6]],

        [[1, 2, 3],
         [4, 5, 6]]])

In [13]:
Y2

tensor([[[ 7,  8],
         [ 7,  8]],

        [[ 9, 10],
         [ 9, 10]]])

In [14]:
Z

tensor([[ 1,  2,  3,  7,  8],
        [ 4,  5,  6,  7,  8],
        [ 1,  2,  3,  9, 10],
        [ 4,  5,  6,  9, 10]])

### END test of product key set with pytorch

In [15]:
a = list(range(10))

In [16]:
a[3] = 7


In [17]:
a

[0, 1, 2, 7, 4, 5, 6, 7, 8, 9]

In [90]:
# def product_key_set(c1,c2,dim1=-2, dim2=-1):
def product_key_set(c1,c2):
    """
    Creates the product key set of two tensors as in https://arxiv.org/abs/1907.05242
    The result is the outer product with respect to the vector concatenation operator of 2 codebooks c1 and c2
    Ideas come from discussion here:
    https://discuss.pytorch.org/t/how-to-create-a-combination-of-concatenations-with-two-tensors/28709/2
    ONLY works on the last 2 dimensions of the codebooks, both codebooks must be the same size
    
    :param c1: first codebook set where dim = N
    :param c2: second codebook set where dim = M
    :return : new tensor dimension NxM (all the other dimensions stay the same)
    """
    #:param dim1: dimension over which the operation must take place
    #:param dim2: dimension over which the operation must take place
    dim1,dim2=-2,-1  # I fix them to avoid issues
    # dim2 = dim1+1
    assert c1.dim() == c2.dim()
#     assert len(c1.shape) > dim1 >= -len(c1.shape) 
#     assert len(c2.shape) > dim2 >= -len(c2.shape) 
#     assert abs(abs(dim1)-abs(dim2)) == 1
#     print(abs( abs(dim1)-abs(dim2)))
    #add the new dimension where cat will work
    
#     print(c1.shape,c2.shape)
    X1 = c1.unsqueeze(dim1)
    Y1 = c2.unsqueeze(dim2)
#     print(X1.shape,Y1.shape)
    #setting dimension to work on
    s1 = [1]*X1.dim()
    s1[dim1]=c1.shape[dim1]
    s2 = [1]*Y1.dim()
    s2[dim2]=c2.shape[dim1]
#     print(s1,s2)
    X2 = X1.repeat(*s1)
    Y2 = Y1.repeat(*s2)
#     print(X1.shape,X2.shape)
    s3 = list(c1.shape)  # [-1] * c1.dim()
    s3[dim2] = c1.shape[dim2] + c2.shape[dim2]  # == Z.shape[dim2]  # ==
    s3[dim1] = c1.shape[dim1] + c2.shape[dim1]
    if len(s3)>2:
        s3[dim1-1] = -1
#     print(s3)
    Z = torch.cat([X2,Y2],dim2)
#     print(Z.shape)
#     Z = Z.reshape(-1,Z.shape[-1])  #here an error
    Z = Z.reshape(*s3)
#     print(Z.shape)
    return Z

    

In [91]:
X.dim()

2

In [92]:
product_key_set(X,Y)

tensor([[ 1,  2,  3,  7,  7],
        [ 1,  2,  3,  8,  8],
        [ 4,  5,  6,  9,  9],
        [ 4,  5,  6, 10, 10]])

In [93]:
x = torch.tensor([[[ [1,2,3], [4,5,6] ]]])
y = torch.tensor([[[ [7, 8], [9, 10] ]]] )

In [94]:
# product_key_set(x,y,1,2)
product_key_set(x,y)

tensor([[[[ 1,  2,  3,  7,  7],
          [ 1,  2,  3,  8,  8],
          [ 4,  5,  6,  9,  9],
          [ 4,  5,  6, 10, 10]]]])